In [ ]:
'''
!pip install python-chess==0.31.2
%ls
!unzip src.zip
'''

In [24]:
import chess
import sys
import torch
import random
from transformers import GPT2LMHeadModel, AutoModel
sys.path.append("src")
from data_utils.chess_tokenizer import ChessTokenizer
import chess.pgn as pgn
import io

2024-07-29 08:36:41.789315: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-29 08:36:41.840910: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [25]:
# Initialize the Model and the Tokenizer

vocab_path = "vocab.txt"
tokenizer = ChessTokenizer(vocab_path)
model = GPT2LMHeadModel.from_pretrained('shtoshni/gpt2-chess-uci')

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [26]:
def get_legal_moves(board):
  legal_moves = set()
  for move in board.legal_moves:
    uci_move = board.uci(move)
    legal_moves.add(uci_move)
  return legal_moves

In [27]:
# input as the PGN of the game. It gets converted in UCI

b = "1. b3 e5 2. Bb2 Nc6 3. e3 d5 4. Bb5 Bd6 5. f4 Qf6 6. Nf3 e4 7. Bxf6 Nxf6 8. Nd4"
#b = "1. Nf3 a6 2. g3 b5 3. Bg2 Bb7 4. O-O c5 5. a4 b4 6. d4 cxd4 7. Nxd4 Bxg2 8. Kxg2 Nf6 9. c3 e6 10. cxb4 Bxb4 11. Bd2 Qb6 12. Nf3 Nc6 13. Nc3 Rb8 14. Be3 Qb7 15. h4 O-O 16. Bg5 Bxc3 17. bxc3 Ne4 18. Qd3 Nxg5 19. hxg5 Ne5 20. Qc2 f5 21. gxf6 Rxf6 22. Rh1 h6 23. Qd1 Rbf8 24. Rb1 Qc6 25. Rf1 Nxf3 26. exf3 Rxf3 27. Kg1 Qc5 28. Kg2 Qc6 29. Kg1 Rxc3 30. Qd4 Qc5 31. Qxc5 Rxc5 32. Rfd1 d5 33. Rb6 Ra5 34. Rxe6 Rxa4 35. Rxd5 Ra1+ 36. Kg2 Ra2 37. Rd7 Raxf2+ 38. Kh3 R2f6"


In [33]:
from datasets import load_dataset

ds = load_dataset("adamkarvonen/chess_sae_individual_games_filtered")
a = ds['train']['text'][4]
b = a[1:300]
print(b)

1.d4 d5 2.c4 e5 3.dxe5 d4 4.f4 Nc6 5.Nf3 Bg4 6.g3 Qd7 7.Kf2 O-O-O 8.Bg2 f6 9.h3 Be6 10.exf6 Nxf6 11.Nbd2 Bc5 12.Kf1 h6 13.a3 a5 14.b3 Rhf8 15.Bb2 g5 16.Ke1 gxf4 17.gxf4 Nh5 18.Ne4 Nxf4 19.Nxc5 Nxg2+ 20.Kd2 Qf7 21.Nxe6 Qxe6 22.Kc1 Qe3+ 23.Kb1 Nf4 24.Re1 d3 25.Bc1 Qe4 26.Nd2 Qf5 27.e3 Ne2 28.Bb2 Rfe8


In [60]:
import random

def chop_game_at_random_point(pgn):
    # Split the PGN string into individual moves
    moves = pgn.split()
    
    # Filter out the move numbers (e.g., "1.", "2.", etc.)
    game_moves = [move for move in moves if not move[0].isdigit()]
    
    # Determine the total number of moves
    total_moves = len(game_moves)
    
    # Ensure we have an even number of moves for simplicity
    if total_moves % 2 != 0:
        total_moves -= 1
    
    # Randomly choose a point to chop, ensuring 50/50 white/black split
    random_index = random.randint(7, total_moves - 1)
    
    # Determine if the random point should be a white or black move
    if random_index % 2 == 0:
        chopped_game = ' '.join(moves[:random_index + 1])
    else:
        chopped_game = ' '.join(moves[:random_index + 2])
    
    return chopped_game




In [41]:
# Example PGN game
pgn_game = "1. Nf3 a6 2. g3 b5 3. Bg2 Bb7 4. O-O c5 5. a4 b4 6. d4 cxd4 7. Nxd4 Bxg2 8. Kxg2 Nf6 9. c3 e6 10. cxb4 Bxb4 11. Bd2 Qb6 12. Nf3 Nc6 13. Nc3 Rb8 14. Be3 Qb7 15. h4 O-O 16. Bg5 Bxc3 17. bxc3 Ne4 18. Qd3 Nxg5 19. hxg5 Ne5 20. Qc2 f5 21. gxf6 Rxf6 22. Rh1 h6 23. Qd1 Rbf8 24. Rb1 Qc6 25. Rf1 Nxf3 26. exf3 Rxf3 27. Kg1 Qc5 28. Kg2 Qc6 29. Kg1 Rxc3 30. Qd4 Qc5 31. Qxc5 Rxc5 32. Rfd1 d5 33. Rb6 Ra5 34. Rxe6 Rxa4 35. Rxd5 Ra1+ 36. Kg2 Ra2 37. Rd7 Raxf2+ 38. Kh3 R2f6"
a = ds['train']['text'][4][1:]
print(a)
# Run the function and print the result
chopped_game = chop_game_at_random_point(pgn_game)
print(chopped_game)

1.d4 d5 2.c4 e5 3.dxe5 d4 4.f4 Nc6 5.Nf3 Bg4 6.g3 Qd7 7.Kf2 O-O-O 8.Bg2 f6 9.h3 Be6 10.exf6 Nxf6 11.Nbd2 Bc5 12.Kf1 h6 13.a3 a5 14.b3 Rhf8 15.Bb2 g5 16.Ke1 gxf4 17.gxf4 Nh5 18.Ne4 Nxf4 19.Nxc5 Nxg2+ 20.Kd2 Qf7 21.Nxe6 Qxe6 22.Kc1 Qe3+ 23.Kb1 Nf4 24.Re1 d3 25.Bc1 Qe4 26.Nd2 Qf5 27.e3 Ne2 28.Bb2 Rfe8 29.Ka2 Rxe3 30.Rf1 Qxh3 31.c5 Ned4 32.Nc4 Re2 33.Rc1 Qg2 34.Rc3 
1. Nf3 a6 2. g3 b5 3. Bg2 Bb7 4. O-O c5 5. a4 b4 6. d4


In [67]:
g = random.randint(0, 90000)
a = ds['train']['text'][g][1:]
b = chop_game_at_random_point(a)

In [68]:
b1 = io.StringIO(b)
game = pgn.read_game(b1)
board = game.board()

move_list = []
for move in game.mainline_moves():
    move_list.append(board.uci(move))
    board.push(move)
b1 = (" ".join(move_list))


#print(board)
#print(game.mainline_moves())
print(b1)

e2e4 g7g6 g1f3 f8g7 b1c3 e7e6 d2d4 g8e7 e4e5 d7d6 f1e2 d6e5 d4e5 d8d1 e2d1 b8c6 e1g1 c6e5 f3e5 g7e5 f1e1 e5g7 c1d2 e8g8 a1b1 b7b6 c3b5 c7c6 d1f3 c8a6 b5c7


In [ ]:
# To make the model predict only 'half' of the move, add initial square of the next move
initial_square = "a2"
b2 = b1 + " " + initial_square
print(b1)
# and put range(1) below  


In [75]:
# here move = the game, can be move = new move (but changing)

def predict_move1(move):

    game_prefix = [tokenizer.bos_token_id]
    game_prefix_str = ""

    game_prefix.extend(tokenizer.encode(move, add_special_tokens=False, get_move_end_positions=False))
    game_prefix_str += move + " "

    greedy_game_prefix = list(game_prefix)
    prefix_tens = torch.tensor([greedy_game_prefix])
    pred_move = ""

    #range was 3 in orig.
    for idx in range(3):
        logits = model(prefix_tens)[0]
        
        # logits is a 3-dimensional tensor with the shape [batch_size, sequence_length, vocab_size].
        # --> Extracting the logits for the last token.
        last_token_logit = logits[0, -1, :]

        token_idx = torch.argmax(last_token_logit).item()
        
        current_token = tokenizer.decode_token(token_idx)
        pred_move += current_token

        if idx == 0 and current_token == tokenizer.eos_token:
            break

        greedy_game_prefix += [token_idx]
        prefix_tens = torch.tensor([greedy_game_prefix])

    if len(pred_move) == 6:
        pred_move = pred_move[:4]

    legal_moves = get_legal_moves(board)
    if pred_move not in legal_moves:
        print("ILLEGAL MOVE")
        print(f"Legal moves: {legal_moves}")

    print(f"LM plays: {pred_move}")

In [74]:
# probing to see if the game provides legal moves given the starting square

def probing_legal_moves1(move0):
    legal_moves = get_legal_moves(board)
    illegal_count = 0
    for leg_move in legal_moves:

        initial_square = leg_move[:2]
        move = move0 + " " + initial_square

        game_prefix = [tokenizer.bos_token_id]
        game_prefix_str = ""

        game_prefix.extend(tokenizer.encode(move, add_special_tokens=False, get_move_end_positions=False))
        game_prefix_str += move + " "

        greedy_game_prefix = list(game_prefix)
        prefix_tens = torch.tensor([greedy_game_prefix])
        pred_move = ""

        for idx in range(1):
            logits = model(prefix_tens)[0]
            
            # logits is a 3-dimensional tensor with the shape [batch_size, sequence_length, vocab_size].
            # --> Extracting the logits for the last token.
            last_token_logit = logits[0, -1, :]

            token_idx = torch.argmax(last_token_logit).item()
            
            current_token = tokenizer.decode_token(token_idx)
            pred_move += current_token

            if idx == 0 and current_token == tokenizer.eos_token:
                break

            greedy_game_prefix += [token_idx]
            prefix_tens = torch.tensor([greedy_game_prefix])

        if len(pred_move) == 6:
            pred_move = pred_move[:4]


        pred_move = initial_square + pred_move
        if pred_move not in legal_moves:
            #print("ILLEGAL MOVE")
            #print(f"Legal moves: {legal_moves}")
            illegal_count += 1

        #print(f"LM plays: {pred_move}")

    print(f"Ratio legal/total: {len(legal_moves)-illegal_count} / {len(legal_moves)}")

In [79]:
g = random.randint(0, 90000)
a = ds['train']['text'][g][1:]
b = chop_game_at_random_point(a)

b1 = io.StringIO(b)
game = pgn.read_game(b1)
board = game.board()

move_list = []
for move in game.mainline_moves():
    move_list.append(board.uci(move))
    board.push(move)
b1 = (" ".join(move_list))

print(b)
print(b1)

move = b1
predict_move1(move)
probing_legal_moves1(move)

1.e4 e6 2.Nf3 g6 3.c3 Bh6 4.Be2 b6 5.O-O g5 6.h3 d6 7.Na3
e2e4 e7e6 g1f3 g7g6 c2c3 f8h6 f1e2 b7b6 e1g1 g6g5 h2h3 d7d6 b1a3
LM plays: a7a6
Ratio legal/total: 26 / 26
